In [1]:
import dspy
import openai
import os
from dotenv import load_dotenv
import datetime
load_dotenv()


True

In [2]:
import GoogleCalendar as gc

2025-08-18 19:28:19.113758+00:00
creating an event.
Event created: https://www.google.com/calendar/event?eid=NXByaDRscGo1OXZ0M205bHZnNm5kZGptb28gc3llc3NzMTMyQG0


In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM(api_key = openai_api_key, model = "gpt-4o-mini")
dspy.settings.configure(lm=lm)

In [4]:
class CalendarAgent(dspy.Signature):
    """
    make calendar events 
    1. assume all events are in UTC
    2. do not add characters to event dates, e.g. "Z" for UTC
    """
    event_details = dspy.InputField(desc="the details of the calendar event")
    current_time = dspy.InputField(desc="the current time")

    event_summary  = dspy.OutputField(desc="the summary of the event")
    event_start = dspy.OutputField(desc="the start time of the event in iso format. utc")
    event_end = dspy.OutputField(desc="the end time of the event in iso format. utc")
    event_location = dspy.OutputField(desc="the location of the event")
    event_description = dspy.OutputField(desc="the description of the event")

In [5]:
class GetEvents(dspy.Signature):
    """
    from the given details make a list of events
    """

    details = dspy.InputField(desc="the details of the events")
    events: list[str] = dspy.OutputField(desc="the list of events")

In [13]:
class Assistant(dspy.Signature):
    """an assistant that can help with tasks. determine the best course of action. ensure excess events arent scheduled"""
    prompt = dspy.InputField(desc="given instructions to follow")
    output = dspy.OutputField(desc="the response to instructions")

In [14]:
def get_upcoming_events():
    """
    Get a list of upcoming events from the calendar.
    """
    client = gc.create_calendar_client()
    events = gc.get_events(client)
    return events

In [15]:
def create_calendar_event(users_input):
    """
    1. pass the users input to the agent.
    2. do not add characters to event dates, e.g. "Z" for UTC
    3. assume all events are upcoming.
    """
    calendar_agent = dspy.ReAct(
        signature=CalendarAgent,
        tools=[]
    )

    event_agent = dspy.Predict(signature = GetEvents)
    event_result = event_agent(details=users_input)
    

    client = gc.create_calendar_client()
    current_time = gc.get_current_time()

    print(f"Event list: {event_result.events}")
    print(f"Current time: {current_time}")

    for event in event_result.events:
        agent_response = calendar_agent(event_details= event, current_time=current_time)

        print(f"\n\nAgent response: \n{agent_response}")

        iso_start_time = datetime.datetime.fromisoformat(agent_response.event_start)
        iso_end_time = datetime.datetime.fromisoformat(agent_response.event_end)
        print(f"\nISO Start Time: {iso_start_time}")


        event = gc.create_event(
            service=client,
            summary=agent_response.event_summary,
            start_time=iso_start_time,
            end_time=iso_end_time,
            location=agent_response.event_location,
            description=agent_response.event_description
        )

In [16]:
react = dspy.ReAct(
    signature=Assistant,
    tools=[create_calendar_event, gc.get_current_time, get_upcoming_events]
)

In [18]:
response = react(prompt="i have art class on monday tuesday and thursday at 3pm till 4:30pm next week starting on the 25th  can you add that?")
print(f"\n{response.output}")

getting events
Event list: ['Art Class on Monday, September 25, 2023 from 15:00 to 16:30', 'Art Class on Tuesday, September 26, 2023 from 15:00 to 16:30', 'Art Class on Thursday, September 28, 2023 from 15:00 to 16:30']
Current time: 2025-08-18 19:35:22.681035+00:00


Agent response: 
Prediction(
    trajectory={'thought_0': 'I have all the necessary information to create the calendar event for the Art Class on September 25, 2023. I will now finalize the task.', 'tool_name_0': 'finish', 'tool_args_0': {}, 'observation_0': 'Completed.'},
    reasoning='The event is an Art Class scheduled for September 25, 2023, from 15:00 to 16:30 UTC. Since the current time is in 2025, this event is in the past. However, I will still provide the details as requested.',
    event_summary='Art Class',
    event_start='2023-09-25T15:00:00',
    event_end='2023-09-25T16:30:00',
    event_location='Not specified',
    event_description='Art Class scheduled for September 25, 2023, from 15:00 to 16:30 UTC.'
)